In [1]:
import time
import umap
import torch
import numpy as np
import pandas as pd
import scanpy as sc
import MASIv2 as masi

import torch.nn as nn

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score

from scipy import sparse

import squidpy as sq

import warnings
warnings.filterwarnings("ignore")

In [137]:
##Slide-seqV2 mouse kidney
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_normal_Puck_200104_01.h5ad"
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_normal_Puck_191204_18.h5ad"
file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_normal_Puck_200104_04.h5ad"
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_autosomal_Puck_200131_20.h5ad"
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_autosomal_Puck_191223_24.h5ad"
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_dkd_Puck_191206_04.h5ad"
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_dkd_Puck_191206_02.h5ad"
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_autosomal_Puck_200210_03.h5ad"
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_autosomal_Puck_200104_07.h5ad"
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_autosomal_Puck_200102_09.h5ad"
#file = "D:/MASIv2_data/Marshall_kidney/Marshall_kidney_autosomal_Puck_200104_10.h5ad"

target = sc.read_h5ad(file)
target.var.index = target.var['feature_name'].values
target.obsm['spatial']=target.obsm['X_spatial']
target.obs['cell_type']=target.obs['author_cell_type']
adata = target.copy()

In [4]:
adata = target['Puck_191109_09']
adata.var.index = adata.var['feature_name'].values

Run MASIv2

In [5]:
markers = pd.read_csv("D:/ReferenceMarkers/mouse_kidney_miao_50markers.csv",header=0,index_col=0)
cell_markers={}
for m in markers.columns:
    cell_markers[m]=markers[m].values.tolist()

In [51]:
marker_list = []
for k, v in cell_markers.items():
    marker_list += v
marker_list = list(set(marker_list))
marker_list = [i for i in marker_list if i in adata.var.index]
len(marker_list)
ad = adata[:,marker_list]

In [30]:
marker_source = []
for k, v in cell_markers.items():
    marker_source += v
marker_source = list(set(marker_source))
marker_source = [i for i in marker_source if i in source.var.index]
len(marker_source)
source = source[:,marker_source]

ref = masi.gene2mat(ad=source,cell_markers=cell_markers,if_tfidf=True,if_thresh=True,thresh=0.9)
ref = ref.reshape(ref.shape[0],ref.shape[1]*ref.shape[2])
scaler = StandardScaler()
ref = scaler.fit_transform(ref)
ref = pd.DataFrame(ref)

ref['cell_type']=source.obs['cell_type'].values
ref = ref.groupby('cell_type').mean()

cellnames = [i for i in cell_markers.keys()]
ref = ref.reindex(cellnames)

ref.fillna(0, inplace=True)

In [52]:
##prepare graph 1.1
ad = adata[:,marker_list].copy()

scores, y_pred = masi.gene2cell(ad=ad,cell_markers=cell_markers,use_weight=False,
                                if_tfidf=True,if_thresh=True,thresh=0.9,use_knn=False)

#sc.pp.neighbors(ad, n_neighbors=10, use_rep='spatial',metric='euclidean',key_added='Adj')#
#adj = ad.obsp['Adj_connectivities']
#sq.gr.spatial_neighbors(ad, n_rings=2, coord_type="grid", n_neighs=8,spatial_key='spatial')
sq.gr.spatial_neighbors(ad, n_rings=2, coord_type="grid", n_neighs=8,spatial_key='X_spatial')
adj = ad.obsp["spatial_connectivities"]

##prepare graph 1.2
#adj = target2_v2.obsp['spatial_connectivities']
#adj = np.array(adj.todense()).flatten()
adj = np.array(adj.todense()).flatten()
src_node = np.repeat([i for i in range(scores.shape[0])],repeats=scores.shape[0])
dst_node = np.tile([i for i in range(scores.shape[0])],scores.shape[0])

edge_data = pd.DataFrame({'Src':src_node,'Dst':dst_node,'Weight':adj})
edge_data = edge_data[edge_data['Weight']>0]

(3771, 11)


In [53]:
mat = masi.gene2mat(ad=ad,cell_markers=cell_markers,if_tfidf=True,if_thresh=True,thresh=0.9)
mat = mat.reshape(mat.shape[0],mat.shape[1]*mat.shape[2])

##either
#graph = masi.process(mat,edge_data)
##or
graph = dgl.graph((edge_data['Src'].values.tolist(), edge_data['Dst'].values.tolist()))
node_features = torch.from_numpy(mat)
graph.ndata['feat'] = node_features

##add self loop
graph = dgl.add_self_loop(graph)

In [54]:
scores, y_pred = masi.gene2cell(ad=ad,cell_markers=cell_markers,use_weight=False,
                                if_tfidf=True,if_thresh=True,thresh=0.9,use_knn=False)

y_pred = y_pred.values

labels = y_pred.copy()
labels = pd.DataFrame(labels)
labels.columns = cell_markers.keys()

(3771, 11)


In [55]:
student = masi.GCNClassifier_noref(output_dim=labels.shape[1],marker_dim=50,t1=0.1,t2=20.0)#5.0 or 2.0 temp
_ = masi.train_noref(graph, student, 200, y_pred)

student.to(torch.device("cpu"))
graph = graph.to(torch.device("cpu"))
student.temp = torch.tensor(1.0, requires_grad=False)

#student.eval()

fea,y_pred = student(graph,graph.ndata['feat'])
fea = torch.Tensor.cpu(fea).detach().numpy()
y_pred = torch.Tensor.cpu(y_pred).detach().numpy()

y_pred = pd.DataFrame(y_pred)
y_pred.columns = cell_markers.keys()
fea = pd.DataFrame(fea)
fea.columns = cell_markers.keys()

labels = y_pred.copy()
scores = fea.copy()

In [56]:
#Label1 accuracy
labels1 = np.argmax(labels.values,axis=1)
knn_pred = []
for i in labels1:
    knn_pred.append(labels.columns[i])
knn_pred = np.array(knn_pred)
annotation = knn_pred.copy()

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, dpi_save=300)
adata.obs['Annotation']=annotation
sc.pl.spatial(adata, color="cell_type_original", spot_size=0.015, frameon=False)
sc.pl.spatial(adata, color="Annotation", spot_size=0.015, frameon=False)